In [1]:
%load_ext cython

In [2]:
import numpy as np

In [33]:
%%cython
# cython: language_level=2
from libc.stdlib cimport malloc

cdef struct struct_coeff:
    int index
    int coeff

cdef class lie_bracket_container:
    """A class efficiently storing the structure coefficients of a Lie algebra, for 
    fast computation of Lie brackets. """
    cdef:
        int max_len
        int num_inds
        int[:,::1] _lengths
        struct_coeff[:,:,::1] _struct_coeffs
        
    def __cinit__(self,max_len,num_inds,structure_coeffs):
        self.max_len = max_len
        self.num_inds = num_inds
        
        cdef int* _length_data = <int*>malloc(num_inds*num_inds*sizeof(int))
        cdef int[:,::1] _lengths = <int[:num_inds,:num_inds]>_length_data
        self._lengths = _lengths
        
        cdef struct_coeff* _data = <struct_coeff*>malloc(
            num_inds*num_inds*max_len*sizeof(struct_coeff)
        )
        cdef struct_coeff[:,:,::1] _struct_coeffs = <struct_coeff[:num_inds, :num_inds, :max_len]>_data
        self._struct_coeffs = _struct_coeffs
    
    def __init__(self,max_len,num_inds,structure_coeffs):
        #self.max_len = max_len
        #self.structure_coeffs = structure_coeffs
        cdef int n,i,j,k,index,coeff
        cdef struct_coeff Cijk
        for (i,j),arr in structure_coeffs:
            l = len(arr)
            self._lengths[i,j]=l
            for k in range(l):
                index, coeff = arr[k]
                Cijk.index = index
                Cijk.coeff = coeff
                self._struct_coeffs[i,j,k]=Cijk
        
    def get_max_len(self):
        return self.max_len
    
    def get_length(self, int i, int j):
        return self._lengths[i,j]
    
    cdef int _get_length(self, int i, int j):
        return self._lengths[i,j]
    
    cdef struct_coeff[:] _get_bracket(self, int i, int j):
        return self._struct_coeffs[i,j]
    
    def get_bracket(self, int i, int j):
        cdef int k,l
        cdef struct_coeff Cijk
        l = self._lengths[i,j]
        arr = []
        for k in range(l):
            Cijk = self._struct_coeffs[i,j,k]
            arr.append((Cijk.index,Cijk.coeff))
        return arr
    
cdef class pbw_element:
    cdef:
        int deg, length
        int[:] coefficients
        int[:] degrees
        int[:,:] monomials
        lie_bracket_container lie_alg
        
    def __cinit__(self, int deg, int length, int[:] coefficients, 
                  int[:] degrees, int[:,:] monomials, lie_bracket_container lie_alg):
        self.deg = deg
        self.length = length
        self.coefficients = coefficients
        self.degrees = degrees
        self.monomials = monomials
        self.lie_alg = lie_alg
        

In [27]:
def make_lie_algebra_container(lie_algebra):
    lie_alg_basis = dict(lie_algebra.basis()).items()
    basis_to_index = {b:i for i,(_,b) in enumerate(lie_alg_basis)}
    index_to_basis = {i:b for b,i in basis_to_index.items()}
    key_to_index = {k:i for i,(k,_) in enumerate(lie_alg_basis)}
    basis_length = len(lie_alg_basis)
    basis_dic = dict()
    for i in range(basis_length):
        for j in range(basis_length):
            brack = index_to_basis[i].bracket(index_to_basis[j])
            mon_coeff_list = []
            for mon,coef in brack.monomial_coefficients().items():
                mon_coeff_list.append((key_to_index[mon],coef))
            basis_dic[(i,j)]=mon_coeff_list
    max_len = max(len(a) for a in basis_dic.values())
    lbc = lie_bracket_container(max_len, basis_length, basis_dic.items())
    return lbc

In [3]:
def get_lie_alg_dict(lie_algebra):
    lie_alg_basis = dict(lie_algebra.basis()).items()
    basis_to_index = {b:i for i,(_,b) in enumerate(lie_alg_basis)}
    index_to_basis = {i:b for b,i in basis_to_index.items()}
    key_to_index = {k:i for i,(k,_) in enumerate(lie_alg_basis)}
    basis_length = len(lie_alg_basis)
    basis_dic = dict()
    for i in range(basis_length):
        for j in range(basis_length):
            brack = index_to_basis[i].bracket(index_to_basis[j])
            mon_coeff_list = []
            for mon,coef in brack.monomial_coefficients().items():
                mon_coeff_list.append((key_to_index[mon],coef))
            basis_dic[(i,j)]=mon_coeff_list
    max_len = max(len(a) for a in basis_dic.values())
    return basis_to_index,index_to_basis,key_to_index,basis_length,basis_dic,max_len
    
def get_structure_coeffs(basis_length,max_len,basis_dic):
    lengths = np.zeros((basis_length,basis_length))
    struct_coeffs = np.zeros((basis_length,basis_length,max_len,2))
    for i in range(basis_length):
        for j in range(basis_length):
            entry = basis_dic[(i,j)]
            lengths[i,j]=len(entry)
            for k,coeff in enumerate(entry):
                struct_coeffs[i,j,k]=coeff
    return max_len, lengths, struct_coeffs

In [21]:
root_sys = 'B3'
def get_root_system(root_sys):
    rl = WeylGroup(root_sys).domain().root_system.root_lattice()
    la = LieAlgebra(QQ, cartan_type = root_sys)
    neg_roots = list(rl.negative_roots())
    def root_sort_key(r):
        vec = -r.to_vector()
        return (sum(vec),tuple(vec))
    neg_roots.sort(key=root_sort_key)
    root_to_index = {r:i for i,r in enumerate(neg_roots)}
    num_roots = len(neg_roots)
    basis = dict(la.basis()).items()
    basis_to_index = {e:root_to_index[k] for k,e in basis if k in root_to_index}
    index_to_basis = {i:e for e,i in basis_to_index.items()}
    brackets = np.zeros((num_roots,num_roots,2),dtype=np.int32)
    for i in range(num_roots):
        for j in range(num_roots):
            brack = index_to_basis[i].bracket(index_to_basis[j]).monomial_coefficients().items()
            if len(brack)==0:
                brackets[i,j]=[-1,0]
            else:
                mon,coeff = brack[0]
                k = root_to_index[mon]
                brackets[i,j] = [k,coeff]
    return num_roots,basis_to_index,index_to_basis,root_to_index,brackets
num_roots,basis_to_index,index_to_basis,root_to_index,struct_coeffs = get_root_system('B3')

In [5]:
struct_coeffs[1,0]

array([3, 1], dtype=int32)

In the `pbw_element` class, having a reference to the Lie algebra is problematic. I think it actually copies it, and it creates a bunch of garbage that I don't know how to collect. Furthermore, even though it has a `cpdef`, `pbw_element` is still a Python object. Now since I don't need that many of them, the instantiation overhead is not such a huge deal, but it does mean that there is no huge point in the `lie_bracket_container` class.

I don't know how to efficiently create C++ classes and use them efficiently with Cython. In fact it seems like it isn't one of the intended features at all, so we best forget about C++ features like classes. 

We best just store the structure coefficients as a numpy array, and feed its memory view + size to a method pbw_element that puts all the monomials in canonical order. 

In fact it seems that since cdef classes are Python objects, we can not work with them at all with the GIL released. Therefore we best just store all the data as numpy arrays and manipulate them using Cython code. We should also think what the best data structure is for such PBW elements. 

I suggest the following algorithm.

We begin with three numpy arrays. One of coefficients, one of lengths and one of indices.
We then store these into C arrays, but allocate more memory than needed so that the arrays can grow. I guess a stack or a linked list works nicely, but anything will do...

Then we iterate the following procedure. We pick a triple (c, l, (X1,...,Xl)). Then for i < l
we check if Xi>Xi+1, if so, we flip Xi, Xi+1 and we compute [Xi,Xi+1] = (k,((Y1,C1),...,(Yk,Ck)))
For each j<k we add (c * Cj,l-1,(X1,...,Xi-1, Yj,Xi+2,...,Xl)) to the end of the stack. 
If Xi==Xi+1 we set c = 0, or delete the entry if it's easy in our data struct (it won't be in a contiguous array for example).

We actually do a bubble sort of the list (X1,...,Xl). See the Wikipedia page for a simple optimization of this algorithm. 

We keep repeating this until we hit the end of the stack.

Then we argsort the list of coefficients, and sort the c and l lists in the same way.

We merge coefficients that have the same value. We should check if there is an efficient way to do that without using pandas or numpy-indexed or whatnot. We could see if pandas is actually notably faster than a simple numpy argsort approach, and if so figure out what kind of algorithm pandas actually uses. 

One other disadvantage of this is that we might end up doing the same computation multiple times. 
We can keep one list for each possible length, and when we produce a new entry put it in the list of the associated lenght and merge it if it already exists, otherwise put it at the end. This extra check shouldn't cost much overhead. That way we never have duplicates so sorting isn't even necessary!

We can still do the same computation multiple times for the maximum length ones, but I don't see an easy way to optimize that. We could count the number of inflexions, and do Bubble on the guy with the most inflexions until there is at least one with the same amount. Then we just step those at the same time and check at every step whether they are the same. The overhead of doing this however greatly exceeds the gains. 

We need to allocate the following amount of memory to the list of length (l-1). We take the product of the following:
- Maximum number of inflexions for a list of length deg: deg^2
- Maximum length of structure coefficient
- elements of the list of length deg elements
- Length of one entry = (deg+1)
- sizeof(int)
To which we add the size of the (l-1) array we already started with. This is a pretty large buffer, but it's acceptable. At most it's going to be:
- deg=20, struct = 5, num entries = 100, sizeof int = 4, so
20^3 * 5 * 100 * 4 = 16MB
In practice of course it will be much less, but I don't think we can sharpen this upper bound very much. Of course we could also take $(|\mathfrak g|+deg-1)!/(deg)!$ but that's bound to be even bigger. We can do better by counting the actual number of inflections, which iirc is O(n log deg), which isn't too terrible.

First it would be useful if we had a reasonable example to work with. Let's get that by just computing prod of two adjacent things in the BGG graph. We also need code to convert the sage format to a np array, which is good to write.

Important observation: We want to do computations in $\mathrm{U}(\mathfrak n)$, not $\mathrm{U}(\mathfrak g)$. This makes the Lie algebra we're interested smaller, but above all it makes `max_len` necessarily equal to 1, since $[f_\alpha,f_\beta] = c_{\alpha,\beta}f_{\alpha+\beta}$ with $c_{\alpha,\beta}$ some coefficient depending only on the root structure (but in a non-trivial way...).
Furthermore there is a (sort of) natural ordering on the roots, which we should take into account. All of this is already done in the BGG class, but better just reimplement it here.

In [7]:
struct_coeffs[0,3]

array([[3., 1.],
       [0., 0.],
       [0., 0.]])

In [8]:
from bggcomplex import *
BGG = BGGComplex("B3")
BGG.find_cycles()
BGG.compute_signs()
maps = BGG.compute_maps(BGG.zero_root)
example_map0 = maps[('1','12')]
example_map1 = maps[('12','123')]
prod = example_map0*example_map1

In [467]:
pbwe = prod
def pbw_to_array(pbwe):
    terms = pbwe.monomial_coefficients().items()
    indices = []
    coeffs = []
    degs = []
    for mon,coeff in terms:
        word_list = mon.to_word_list()
        indices.append(
            np.array([root_to_index[k] for k in word_list],dtype=np.int32)
        )
        degs.append(len(word_list))
        coeffs.append(coeff)
    max_deg = max(degs)
    pbw_array = np.zeros((len(terms),max_deg+2),dtype=np.int32)-1
    pbw_array[:,0] = np.array(degs,dtype=np.int32)
    pbw_array[:,1] = np.array(coeffs,dtype=np.int32)
    for i,l,a in zip(range(len(terms)),degs,indices):
        pbw_array[i,2:l+2]=a
    return pbw_array

In [469]:
pbw_array0 = pbw_to_array(example_map0)
pbw_array1 = pbw_to_array(example_map1)
pbw_array2 = pbw_to_array(example_map0*example_map1)

In [477]:
pbw_array0 = pbw_to_array(example_map0)
pbw_array1 = pbw_to_array(example_map1)
pbw_array = compute_product(pbw_array0,pbw_array1)

In [552]:
%%cython
#cython: language_level = 2 
#cython: boundscheck=False
#cython: wraparound=False

import numpy as np
with nogil:
    cimport numpy as np
from cython.parallel cimport prange

cdef int[:,:] compute_product(int[:,:] pbw_array0, int[:,:] pbw_array1) nogil:    
    cdef:
        int len0 = len(pbw_array0)
        int len1 = len(pbw_array1)
        int[:, :] pbw_array2 
    with gil:
        pbw_array2 = np.zeros((len0*len1,pbw_array0.shape[1]+pbw_array1.shape[1]-2),dtype=np.int32)-1
    
    cdef int i, j, k, deg0, deg1
    for i in range(len0):
        for j in range(len1):
            k = i*len1+j
            deg0 = pbw_array0[i,0]
            deg1 = pbw_array1[j,0]
            pbw_array2[k,0]=deg0+deg1
            pbw_array2[k,1]=pbw_array0[i,1]*pbw_array1[j,1] #column 1 is coeff of monomial
            
            #new monomial is concat of two monomials. 
            pbw_array2[k,2:deg0+2]=pbw_array0[i,2:deg0+2]
            pbw_array2[k,deg0+2:deg0+deg1+2]=pbw_array1[j,2:deg1+2]
    return pbw_array2

cdef int[:,:] pbw_order(int[:,:] pbw_array0, int[:,:,:] struct_coeffs) nogil:
    #allocate an amount of memory that is *probably* enough. If not, then detect this at runtime
    #and copy everything to an array, say, 4 times the size. 
    #We should count how long the array is in practical cases to improve on this heuristic.
        
    cdef int max_size
    with gil:
        max_size = pbw_array0.shape[0]*(pbw_array0.shape[1]+5)+2
        max_size = 2**int(np.ceil(np.log(max_size)/np.log(2)))

    #initialize arrays
    cdef int max_deg = pbw_array0.shape[1]-2
    cdef int[:,:] pbw_array
    with gil:
        pbw_array = np.zeros((max_size,max_deg+2),dtype=np.int32)-1
    
    #populate them partially
    cdef int i_max = len(pbw_array0)
    
    pbw_array[:i_max,:] = pbw_array0
    
    cdef int deg,i,j
    cdef int temp = 0
    cdef int[:] row, new_row
    cdef int num_swaps
    cdef int[:] bracket
    for i in range(max_size):
        row = pbw_array[i]
        deg = row[0]
        num_swaps = 1
        while num_swaps>0:
            num_swaps = 0 
            for j in range(2,deg+1): #indices start at column 2
                if row[j]>row[j+1]: #bubble sort condition
                    
                    #compute Lie bracket of row[j], row[j+1]
                    bracket = struct_coeffs[row[j],row[j+1]]

                    #if row[j], row[j+1] do not commute, 
                    #copy the row to the end of the array, with
                    #degree lowered by one and row[j], row[j+1] 
                    #swapped out by [row[j],row[j+1]]
                    if bracket[0]!=-1:
                        with gil:
                            new_row = np.zeros(max_deg+2,dtype=np.int32)-1
                        new_row[0] = deg-1
                        new_row[1] = row[1]*bracket[1]
                        new_row[2:j]=row[2:j]
                        new_row[j]=bracket[0]
                        new_row[j+1:deg+1]=row[j+2:deg+2]
                        
                        #strore result at end of array, and increase size by one
                        pbw_array[i_max]=new_row
                        i_max+=1

                    #swap entries j, j+1
                    temp = row[j+1]
                    row[j+1]=row[j]
                    row[j]=temp
                    
                    #if no swaps happened, list is sorted
                    num_swaps+=1
        if i>i_max:
            break
    pbw_array = pbw_array[:i_max]
    
    return pbw_array

cdef int[:] merge_argsort(int[:,:] array, int num_rows, int num_cols) nogil:
    cdef int[:] inds, work
    with gil:
        inds = np.arange(num_rows,dtype=np.int32)
        work = np.arange(num_rows,dtype=np.int32)
    split_merge(array,work,inds,0,num_rows,num_cols)
    return inds
    
cdef void split_merge(int[:,:] array, int[:] work, int[:] inds, 
                      int i_start, int i_stop, int num_cols) nogil:
    if(i_stop-i_start<2):
        return
    i_mid = (i_stop+i_start)//2
    split_merge(array,inds,work,i_start,i_mid,num_cols)
    split_merge(array,inds,work,i_mid,i_stop,num_cols)
    merge(array,work,inds,i_start,i_mid,i_stop,num_cols)

cdef void merge(int[:,:] array,int[:] inds,int[:] work, int i_start,
                int i_mid,int i_stop,int num_cols) nogil:
    cdef int i = i_start
    cdef int j = i_mid
    #we alternate between two arrays as a buffer to avoid copying!
    
    cdef int k
    for k in range(i_start,i_stop):
        if (i<i_mid) and ((j>=i_stop) or (row_leq(array[inds[i]],array[inds[j]],num_cols))):
            work[k]=inds[i]
            i+=1
        else:
            work[k]=inds[j]
            j+=1
    
cdef int row_leq(int[:] row0,int[:] row1,int num_cols) nogil:
    cdef int l
    for l in range(num_cols):
        if row0[l]<row1[l]:
            return 1
        elif row1[l]<row0[l]:
            return 0
    else: #all entries the same, return True
        return 1

cdef int row_isequal(int[:] row0, int[:] row1,int num_cols) nogil:
    cdef int l
    for l in range(num_cols):
        if row0[l]!=row1[l]:
            return 0
    else:
        return 1
    
cdef int[:,:] merge_reduce(int[:,:] pbw_array) nogil:
    cdef:
        int num_cols, num_rows
        int[:] argsort
        int[:,:] new_array
        
    num_cols = pbw_array.shape[1]-2
    num_rows = pbw_array.shape[0]
    with gil:
        new_array = np.zeros_like(pbw_array)
    
    argsort = merge_argsort(pbw_array[:,2:],num_rows,num_cols)

    new_array[0]=pbw_array[argsort[0]]
    cdef int i = 0
    cdef int j = 0
    for i in range(1,num_rows):
        if row_isequal(pbw_array[argsort[i-1],2:],pbw_array[argsort[i],2:],num_cols):
            new_array[j,1]+=pbw_array[argsort[i],1]
        else:
            j+=1
            new_array[j]=pbw_array[argsort[i]]
    j+=1
    return new_array[:j]

cpdef int[:,:] pbw_product(int[:,:] pbw_array0, int[:,:] pbw_array1,int[:,:,:] struct_coeffs) nogil:
    return merge_reduce(pbw_order(
        compute_product(pbw_array0, pbw_array1), struct_coeffs
    ))

In [555]:
%timeit -n 100000
pbw_product_loop(pbw_array0,pbw_array1,struct_coeffs)

KeyboardInterrupt: 

Seems the operation is memory bound, so there's virtually no speed up when run in parallel. That's cool....

In [558]:
%timeit -n 10000 example_map0*example_map1

10000 loops, best of 3: 287 µs per loop
